# Imports

In [19]:
import pandas as pd
import pyspark
import string
import time

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import col, isnull, sum, split, when

# Data preparation (Pandas)

In [5]:
#Load the dataset
start = time.time()

df = pd.read_csv("originalData.csv")

end = time.time()

print('Demorou ', (end - start), " seconds")

NameError: name 'start_time' is not defined

In [3]:
#Drop the column "views", "features" and "id"
df.drop(["views","features","id"], axis=1, inplace=True)

In [4]:
#Check how many nulls have column "lyrics"
null_lyrics = df["lyrics"].isnull().sum()
print("Number of 'nulls' in the 'lyrics' column: " + str(null_lyrics))

#Drop the rows that have nulls values in the "lyrics" column
df.dropna(subset=['lyrics'], inplace=True)

Number of 'nulls' in the 'lyrics' column: 2132


In [5]:
#Check how many different years are in column "year"
year_count = df['year'].value_counts()

#Drop the years before 1900
df = df[df["year"] >= 1900]

#Drop the years after 2022
df = df[df["year"] <= 2022]

In [6]:
#Check how many nulls have column "title"
null_title = df["title"].isnull().sum()
print("Number of 'nulls' in the 'title' column: " + str(null_title))

#Drop the rows that have nulls values in the "title" column
df.dropna(subset=['title'], inplace=True)

Number of 'nulls' in the 'title' column: 331


In [7]:
#Check how many nulls have column "title"
null_title = df["title"].isnull().sum()
print("Number of 'nulls' in the 'title' column: " + str(null_title))

#Check how many nulls have column "tag"
null_tag = df["tag"].isnull().sum()
print("Number of 'nulls' in the 'tag' column: " + str(null_tag))

#Check how many nulls have column "artist"
null_artist = df["artist"].isnull().sum()
print("Number of 'nulls' in the 'artist' column: " + str(null_artist))

#Check how many nulls have column "year"
null_year = df["year"].isnull().sum()
print("Number of 'nulls' in the 'year' column: " + str(null_year))

#Check how many nulls have column "lyrics"
null_lyrics = df["lyrics"].isnull().sum()
print("Number of 'nulls' in the 'lyrics' column: " + str(null_lyrics))

Number of 'nulls' in the 'title' column: 0
Number of 'nulls' in the 'tag' column: 0
Number of 'nulls' in the 'artist' column: 0
Number of 'nulls' in the 'year' column: 0
Number of 'nulls' in the 'lyrics' column: 0


In [8]:
df

,title,tag,artist,year,lyrics
0,Killa Cam,rap,Cam'ron,2004,"[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki..."
1,Can I Live,rap,JAY-Z,1996,"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,..."
2,Forgive Me Father,rap,Fabolous,2003,Maybe cause I'm eatin\nAnd these bastards fien...
3,Down and Out,rap,Cam'ron,2004,[Produced by Kanye West and Brian Miller]\n\n[...
4,Fly In,rap,Lil Wayne,2005,"[Intro]\nSo they ask me\n""Young boy\nWhat you ..."
...,...,...,...,...,...
5913406,Raise Our Hands,pop,"Culture Code, Pag & Mylo",2016,[Verse 1]\nHere our purpose feels alive\nWe ar...
5913407,CEO,rap,Antropolita,2022,Jestem CEO w tym\nTo jara twoją bitch\nNikt na...
5913408,JokAir - Big Drip ft. ARÓN LineMa Traducción ...,rap,Genius Traducciones al Espaol,2022,[Instrumental]
5913409,New Number,country,Alana Springsteen,2022,"[Verse 1]\nYou need a new number, one that ain..."


In [ ]:
#Cleaning the column "lyrics"
df["lyrics"] = df["lyrics"].str.lower()
df["lyrics"] = df["lyrics"].str.lower().str.replace(r"instrumental|intro|guitar|solo","")
df["lyrics"] = df["lyrics"].str.replace("\n"," ").str.replace(r"[^\w\d'\s]+","").str.replace("efil ym fo flah","")
df["lyrics"] = df["lyrics"].str.strip()

# Data Preparation (Pyspark)

In [2]:
#Load the dataset

start = time.time()

# Create a SparkSession
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

# Read the CSV file into a DataFrame
df0 = spark.read.format("csv").option("header", "true").load("originalData.csv")

end = time.time()

print('Demorou ', (end - start), " seconds")

Demorou  5.413204908370972  seconds


In [3]:
#Drop the column "views", "features" and "id"
df0 = df0.drop(*["id", "features", "views"])

In [4]:
#Drop the rows that have nulls values in the "lyrics" column
df0 = df0.na.drop(subset=["lyrics"])

In [6]:
#Drop the years before 1900 and after 2022
df0 = df0.filter((df0.year >= 1900) & (df0.year <= 2022))

In [13]:
#Drop the rows that have nulls values in the "title" column
df0 = df0.na.drop(subset=["title"])

In [14]:
#Drop the rows that have nulls values in the "tag" column
df0 = df0.na.drop(subset=["tag"])

In [18]:
# Create a new column with Boolean values indicating whether each row is null or not
df_with_nulls = df0.select([sum(col(c).isNull().cast("int")).alias(c) for c in df0.columns])

# Display the result
df_with_nulls.show()

+-----+---+------+----+------+
|title|tag|artist|year|lyrics|
+-----+---+------+----+------+
|    0|  0|     0|   0|     0|
+-----+---+------+----+------+



In [20]:
df0 = df0.withColumn("decade", when((df0.year >= 1900) & (df0.year < 1910), "00s")
                              .when((df0.year >= 1910) & (df0.year < 1920), "10s")
                              .when((df0.year >= 1920) & (df0.year < 1930), "20s")
                              .when((df0.year >= 1930) & (df0.year < 1940), "30s")
                              .when((df0.year >= 1940) & (df0.year < 1950), "40s")
                              .when((df0.year >= 1950) & (df0.year < 1960), "50s")
                              .when((df0.year >= 1960) & (df0.year < 1980), "60s")
                              .when((df0.year >= 1970) & (df0.year < 1980), "70s")
                              .when((df0.year >= 1980) & (df0.year < 1990), "80s")
                              .when((df0.year >= 1990) & (df0.year < 2000), "90s")
                              .when((df0.year >= 2000) & (df0.year < 2010), "000s")
                              .when((df0.year >= 2010) & (df0.year < 2020), "010s")
                              .when(df0.year >= 2020, "020s")
                             .otherwise("unknown"))

In [21]:
df0.show()

+--------------------+---+---------+----+--------------------+------+
|               title|tag|   artist|year|              lyrics|decade|
+--------------------+---+---------+----+--------------------+------+
|           Killa Cam|rap|  Cam'ron|2004|   ""Opera Steve""}"|  000s|
|          Can I Live|rap|    JAY-Z|1996|[Produced by Irv ...|   90s|
|   Forgive Me Father|rap| Fabolous|2003|Maybe cause I'm e...|  000s|
|        Down and Out|rap|  Cam'ron|2004|      ""Kanye West""|  000s|
|              Fly In|rap|Lil Wayne|2005|             [Intro]|  000s|
|      Lollipop Remix|rap|Lil Wayne|2008|  ""Static Major""}"|  000s|
|          Im Not You|rap|   Clipse|2002|""Roscoe P. Coldc...|  000s|
|         Family Ties|rap|  Cam'ron|2004|     ""Lady Wray""}"|  000s|
|   Rockin and Rollin|rap|  Cam'ron|1998|           [Verse 1]|   90s|
|       Lord You Know|rap|  Cam'ron|2004|   ""Juelz Santana""|  000s|
|    Money On My Mind|rap|Lil Wayne|2005|             [Intro]|  000s|
|     Think Yall Kno

# Data Visualization

In [3]:
data.head()

,title,tag,artist,year,views,features,lyrics,id
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6


In [6]:
data.tail()

,title,tag,artist,year,views,features,lyrics,id
5913406,Raise Our Hands,pop,"Culture Code, Pag & Mylo",2016,3,"{Elex,""Culture Code / Pag & Mylo""}",[Verse 1]\nHere our purpose feels alive\nWe ar...,7882845
5913407,CEO,rap,Antropolita,2022,5,{},Jestem CEO w tym\nTo jara twoją bitch\nNikt na...,7882846
5913408,JokAir - Big Drip ft. ARÓN LineMa Traducción ...,rap,Genius Traducciones al Espaol,2022,1,"{""Genius Traducciones al Español""}",[Instrumental],7882847
5913409,New Number,country,Alana Springsteen,2022,1,{},"[Verse 1]\nYou need a new number, one that ain...",7882848
5913410,Gone,rap,VAPO,2022,2,"{""Loco (로꼬)""}",[Instrumental],7882849


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5913411 entries, 0 to 5913410
Data columns (total 8 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   title     object
 1   tag       object
 2   artist    object
 3   year      int64 
 4   views     int64 
 5   features  object
 6   lyrics    object
 7   id        int64 
dtypes: int64(3), object(5)
memory usage: 360.9+ MB


In [8]:
data.describe()

,year,views,id
count,5.913411e+06,5.913411e+06,5.913411e+06
mean,2.010663e+03,2.670159e+03,4.028247e+06
std,4.430131e+01,4.409858e+04,2.296420e+06
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,2.010000e+03,1.600000e+01,1.821168e+06
50%,2.016000e+03,6.400000e+01,4.161274e+06
75%,2.019000e+03,3.490000e+02,6.007914e+06
max,2.100000e+03,2.335142e+07,7.882849e+06


In [10]:
data_tag = data['tag'].value_counts()
data_tag

pop        2519256
rap        1962010
rock        892220
rb          225342
misc        208714
country     105869
Name: tag, dtype: int64

In [11]:
data_year = data['year'].value_counts()
data_year

2020    683728
2021    630730
2019    576789
2015    521246
2018    481585
         ...  
1100         1
1437         1
936          1
1769         1
615          1
Name: year, Length: 747, dtype: int64